In [ ]:
from neo4j import GraphDatabase

# Connexion à Neo4j locale
uri = "bolt://localhost:7687"  # Utilisation de l'URI locale
username = "neo4j"  # Nom d'utilisateur par défaut
password = "password"  # Mot de passe que tu as défini lors de l'installation

# Créer un driver Neo4j
driver = GraphDatabase.driver(uri, auth=(username, password))

# Fonction pour charger les films avec LOAD CSV (création de nœuds Movie)
def create_movies():
    with driver.session() as session:
        query = """
        LOAD CSV WITH HEADERS FROM 'file:///movies.csv' AS row
        CREATE (m:Movie {
            movieId: toInteger(row.movieId),
            title: row.title,
            genres: row.genres
        })
        """
        session.run(query)
        print("Movies created.")

# Fonction pour charger les liens IMDB et TMDB avec links.csv
def create_links():
    with driver.session() as session:
        query = """
        LOAD CSV WITH HEADERS FROM 'file:///links.csv' AS row
        MATCH (m:Movie {movieId: toInteger(row.movieId)})
        SET m.imdbId = toInteger(row.imdbId), m.tmdbId = toInteger(row.tmdbId)
        """
        session.run(query)
        print("Links loaded (IMDB and TMDB).")

# Fonction pour créer les utilisateurs avec des nœuds User
def create_users():
    with driver.session() as session:
        query = """
        LOAD CSV WITH HEADERS FROM 'file:///ratings.csv' AS row
        MERGE (u:User {userId: toInteger(row.userId)})
        """
        session.run(query)
        print("Users created.")

# Fonction pour créer les tags avec des nœuds Tag
def create_tags():
    with driver.session() as session:
        query = """
        LOAD CSV WITH HEADERS FROM 'file:///genome-tags.csv' AS row
        CREATE (:Tag {tagId: toInteger(row.tagId), tag: row.tag})
        """
        session.run(query)
        print("Tags created.")

# Fonction pour charger les relations d'évaluation avec LOAD CSV (création de relations RATED)
def create_rated_relationships():
    with driver.session() as session:
        query = """
        LOAD CSV WITH HEADERS FROM 'file:///ratings.csv' AS row
        MATCH (m:Movie {movieId: toInteger(row.movieId)})
        MATCH (u:User {userId: toInteger(row.userId)})
        CREATE (u)-[:RATED {rating: toFloat(row.rating), timestamp: toInteger(row.timestamp)}]->(m)
        """
        session.run(query)
        print("RATED relationships created.")

# Fonction pour charger les relations de tags avec LOAD CSV (création de relations TAGGED)
def create_tagged_relationships():
    with driver.session() as session:
        query = """
        LOAD CSV WITH HEADERS FROM 'file:///tags.csv' AS row
        MATCH (m:Movie {movieId: toInteger(row.movieId)})
        MATCH (u:User {userId: toInteger(row.userId)})
        CREATE (u)-[:TAGGED {tag: row.tag, timestamp: toInteger(row.timestamp)}]->(m)
        """
        session.run(query)
        print("TAGGED relationships created.")

# Fonction pour charger les scores de pertinence (HAS_TAG relations)
def create_has_tag_relationships():
    with driver.session() as session:
        query = """
        LOAD CSV WITH HEADERS FROM 'file:///genome-scores.csv' AS row
        MATCH (m:Movie {movieId: toInteger(row.movieId)})
        MATCH (t:Tag {tagId: toInteger(row.tagId)})
        CREATE (m)-[:HAS_TAG {relevance: toFloat(row.relevance)}]->(t)
        """
        session.run(query)
        print("HAS_TAG relationships created.")


# Charger les fichiers CSV dans Neo4j
def load_data():
    # Étape 1 : Créer les nœuds
    create_movies()  # Créer les nœuds de films
    create_links()  # Charger les liens IMDB et TMDB
    create_users()   # Créer les nœuds d'utilisateurs
    # create_tags()    # Créer les nœuds de tags

    # Étape 2 : Créer les relations
    create_rated_relationships()  # Créer les relations RATED
    create_tagged_relationships()  # Créer les relations TAGGED
    # create_has_tag_relationships()  # Créer les relations HAS_TAG

# Exécuter l'import des données
load_data()

# Fermer la connexion Neo4j
driver.close()
print("Connexion à Neo4j fermée.")



### tous les film avec leur relations
MATCH (m:Movie)
OPTIONAL MATCH (m)<-[:TAGGED]-(u:User)  // Récupère les tags associés aux films via les utilisateurs
OPTIONAL MATCH (m)<-[r:RATED]-(u2:User)  // Récupère les évaluations pour chaque film
WITH m, collect(DISTINCT u.tag) AS tags, avg(r.rating) AS AverageRating  // Calcule la moyenne des évaluations et collecte les tags
RETURN m.title AS MovieTitle, 
       tags, 
       split(m.genres, '|') AS GenresArray,  // Divise la chaîne de genres en tableau
       coalesce(AverageRating, 0) AS AverageRating  // Utilise '0' s'il n'y a pas d'évaluations
LIMIT 25


### tous les film qui ont des tags avec la moyenne rating
MATCH (m:Movie)
OPTIONAL MATCH (m)<-[t:TAGGED]-(u:User)  // Récupère les relations TAGGED
OPTIONAL MATCH (m)<-[r:RATED]-(u2:User)  // Récupère les évaluations pour chaque film
WITH m, collect(DISTINCT t.tag) AS tags, avg(r.rating) AS AverageRating  // Calcule la moyenne des évaluations et collecte les tags
WHERE size(tags) > 0  // Filtre pour ne récupérer que les films qui ont des tags
RETURN m.title AS MovieTitle, 
       m.movieId,
       tags, 
       split(m.genres, '|') AS GenresArray,  // Divise la chaîne de genres en tableau
       coalesce(AverageRating, 0) AS AverageRating  // Utilise '0' s'il n'y a pas d'évaluations
LIMIT 25

## Requête pour afficher tous les types de tags :

MATCH (m:Movie)<-[t:TAGGED]-(u:User)
RETURN DISTINCT t.tag AS Tag
ORDER BY Tag


## Requête de recherche par tag spécifique :

MATCH (m:Movie)<-[t:TAGGED]-(u:User)
WHERE t.tag = 'Cold War'  // Remplacez 'votre_tag_ici' par le tag que vous recherchez
OPTIONAL MATCH (m)<-[r:RATED]-(u2:User)
WITH m, collect(DISTINCT t.tag) AS tags, avg(r.rating) AS AverageRating
RETURN m.title AS MovieTitle, 
       m.movieId, 
       tags, 
       split(m.genres, '|') AS GenresArray,  // Divise la chaîne de genres en tableau
       coalesce(AverageRating, 0) AS AverageRating  // Utilise '0' s'il n'y a pas d'évaluations
LIMIT 25


## Requête pour afficher tous les genres distincts :

MATCH (m:Movie)
WITH m, split(m.genres, '|') AS genreList  // Décompose les genres
UNWIND genreList AS genre  // Sépare chaque genre en une ligne distincte
RETURN DISTINCT genre  // Récupère uniquement les genres distincts
ORDER BY genre


## Requête pour rechercher des films avec plusieurs genres :

MATCH (m:Movie)
WHERE ALL(genre IN ['Action', 'Adventure'] 
          WHERE genre IN split(m.genres, '|'))  // Vérifie si tous les genres spécifiés sont présents
RETURN m.title AS MovieTitle, 
       split(m.genres, '|') AS GenresArray  // Divise la chaîne de genres en tableau


## Requête pour Afficher un film et afficher toutes ses informations :

MATCH (m:Movie {movieId: 1})  // Remplacer '1' par l'ID du film à rechercher
OPTIONAL MATCH (m)<-[r:RATED]-(u:User)  // Récupère les évaluations pour le film
OPTIONAL MATCH (m)<-[t:TAGGED]-(u2:User)  // Récupère les tags associés au film par utilisateur
WITH m, u.userId AS UserId, r.rating AS Rating, collect(t.tag) AS Tags  // Collecte les tags associés pour chaque utilisateur
WITH m, collect({
    userId: UserId,  // Crée un objet utilisateur
    rating: Rating,  // Inclut la note donnée
    tags: Tags       // Inclut les tags associés
}) AS UsersInfo
RETURN m.movieId AS MovieID, 
       m.title AS Title, 
       split(m.genres, '|') AS Genres,  // Affiche les genres en tableau
       m.imdbId AS IMDB_ID, 
       m.tmdbId AS TMDB_ID,
       UsersInfo[0..25] AS PaginatedUsersInfo  // Pagination manuelle avec plage d'index



## Requête Cypher pour rechercher des films contenant une séquence de caractères dans leur titre :

MATCH (m:Movie)
WHERE toLower(m.title) CONTAINS toLower('toy')
RETURN m.movieId AS MovieID, 
       m.title AS Title, 
       split(m.genres, '|') AS Genres,
       m.imdbId AS IMDB_ID, 
       m.tmdbId AS TMDB_ID
LIMIT 25


In [5]:
import requests

# URL pour l'authentification et l'obtention d'une session
auth_url = "https://movielens.org/api/sessions"
auth_data = {
    "userName": "atf202",
    "password": "Bouboule12345"
}

# Créer une session pour gérer les cookies
session = requests.Session()

# Effectuer l'appel POST pour obtenir l'authentification avec gestion des cookies
response = session.post(auth_url, json=auth_data)

if response.status_code == 200:
    # L'authentification a réussi, les cookies sont automatiquement stockés dans la session
    print("Authentification réussie")

    # URL pour récupérer les informations sur un film
    movie_url = "https://movielens.org/api/movies/1"

    # Effectuer l'appel GET avec la session pour récupérer les informations du film
    movie_response = session.get(movie_url)

    if movie_response.status_code == 200:
        movie_info = movie_response.json()
        print("Film Information: ", movie_info)
    else:
        print(f"Échec de la récupération des informations du film: {movie_response.status_code}")
else:
    print(f"Échec de l'authentification: {response.status_code}")


Authentification réussie
Film Information:  {'status': 'success', 'data': {'movieDetails': {'movieId': 1, 'rec_val_type': '', 'tstamp': None, 'movie': {'movieId': 1, 'tmdbMovieId': 862, 'imdbMovieId': '0114709', 'title': 'Toy Story', 'originalTitle': 'Toy Story', 'mpaa': 'G', 'runtime': 81, 'releaseDate': '1995-10-30', 'dvdReleaseDate': '2001-03-20', 'genres': ['Animation', 'Adventure', 'Family', 'Comedy'], 'languages': ['English'], 'directors': ['John Lasseter'], 'actors': ['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim Varney', 'Wallace Shawn', 'John Ratzenberger', 'Annie Potts', 'John Morris', 'Erik von Detten', 'Laurie Metcalf', 'R. Lee Ermey', 'Sarah Freeman', 'Penn Jillette', 'Jack Angel', 'Spencer Aste', 'Greg Berg', 'Lisa Bradley', 'Kendall Cunningham', 'Debi Derryberry', 'Cody Dorkin', 'Bill Farmer', 'Craig Good', 'Gregory Grudt', 'Danielle Judovits', 'Sam Lasseter', 'Brittany Levenbrown', 'Sherry Lynn', 'Scott McAfee', 'Mickie McGowan', "Ryan O'Donohue", 'Jeff Pidgeon', 'Patri

_______________________________________________________________________________________________________________________________________________________________

## A rajouté au future

In [10]:
# import requests
# from neo4j import GraphDatabase

# # Connexion à Neo4j locale
# uri = "bolt://localhost:7687"  # Utilisation de l'URI locale
# username = "neo4j"
# password = "password"

# # Créer un driver Neo4j
# print("Initializing Neo4j connection...")
# driver = GraphDatabase.driver(uri, auth=(username, password))
# print("Neo4j connection initialized successfully.")

# # Fonction pour récupérer les films par blocs depuis Neo4j
# def get_movies_from_neo4j(batch_size=1000, page=0):
#     print(f"Fetching movies from Neo4j, page {page} with batch size {batch_size}...")
#     with driver.session() as session:
#         result = session.run("""
#         MATCH (m:Movie)
#         RETURN m.movieId AS movieId
#         SKIP $skip LIMIT $limit
#         """, skip=page * batch_size, limit=batch_size)
#         movies = result.data()
#         print(f"Fetched {len(movies)} movies from page {page}.")
#         return movies

# # Fonction pour mettre à jour les films avec les informations obtenues depuis l'API
# def create_or_update_movie_in_neo4j(movie_data):
#     with driver.session() as session:
#         query = """
#         MERGE (m:Movie {movieId: $movieId})
#         SET m.title = $title,
#             m.genres = $genres,
#             m.imdbId = $imdbId,
#             m.tmdbId = $tmdbId,
#             m.posterPath = $posterPath,
#             m.backdropPaths = $backdropPaths,
#             m.plotSummary = $plotSummary,
#             m.releaseDate = $releaseDate,
#             m.youtubeTrailerIds = $youtubeTrailerIds
#         """
#         session.run(query, movieId=movie_data["movieId"],
#                     title=movie_data["title"],
#                     genres="|".join(movie_data["genres"]),
#                     imdbId=movie_data["imdbMovieId"],
#                     tmdbId=movie_data["tmdbMovieId"],
#                     posterPath=f"https://image.tmdb.org/t/p/original{movie_data['posterPath']}",
#                     backdropPaths="|".join([f"https://image.tmdb.org/t/p/original{path}" for path in movie_data["backdropPaths"]]),
#                     plotSummary=movie_data["plotSummary"],
#                     releaseDate=movie_data["releaseDate"],
#                     youtubeTrailerIds="|".join(movie_data["youtubeTrailerIds"]))
#         # Pas de print ici pour chaque film pour éviter un log trop verbeux.

# # Fonction pour créer les relations acteurs et réalisateurs
# def create_actors_and_directors_in_neo4j(movie_id, actors, directors):
#     with driver.session() as session:
#         for actor in actors:
#             session.run("""
#             MERGE (a:Actor {name: $actorName})
#             MERGE (m:Movie {movieId: $movieId})
#             MERGE (a)-[:ACTED_IN]->(m)
#             """, actorName=actor, movieId=movie_id)
#         for director in directors:
#             session.run("""
#             MERGE (d:Director {name: $directorName})
#             MERGE (m:Movie {movieId: $movieId})
#             MERGE (d)-[:DIRECTED]->(m)
#             """, directorName=director, movieId=movie_id)
#         # Pas de print pour chaque acteur/réalisateur pour éviter un log trop verbeux.

# # Fonction pour récupérer les détails du film à partir de l'API MovieLens
# def get_movie_details_from_movielens_api(movie_id):
#     auth_url = "https://movielens.org/api/sessions"
#     auth_data = {"userName": "atf202", "password": "Bouboule12345"}
    
#     # Créer une session pour les requêtes avec cookies
#     session = requests.Session()
#     response = session.post(auth_url, json=auth_data)
    
#     if response.status_code == 200:
#         movie_url = f"https://movielens.org/api/movies/{movie_id}"
#         movie_response = session.get(movie_url)
        
#         if movie_response.status_code == 200:
#             movie_data = movie_response.json()["data"]["movieDetails"]["movie"]
#             return {
#                 "movieId": movie_data["movieId"],
#                 "title": movie_data["title"],
#                 "genres": movie_data["genres"],
#                 "imdbMovieId": movie_data["imdbMovieId"],
#                 "tmdbMovieId": movie_data["tmdbMovieId"],
#                 "posterPath": movie_data["posterPath"],
#                 "backdropPaths": movie_data["backdropPaths"],
#                 "plotSummary": movie_data["plotSummary"],
#                 "releaseDate": movie_data["releaseDate"],
#                 "youtubeTrailerIds": movie_data.get("youtubeTrailerIds", []),
#                 "actors": movie_data.get("actors", []),  # Récupère les acteurs
#                 "directors": movie_data.get("directors", [])  # Récupère les réalisateurs
#             }
#         else:
#             print(f"Failed to fetch movie details for movie ID {movie_id}. Status code: {movie_response.status_code}")
#             return None
#     else:
#         print(f"Failed to authenticate with MovieLens API. Status code: {response.status_code}")
#         return None

# # Fonction principale pour lire les films par lots et mettre à jour Neo4j
# def update_movies_with_api_details():
#     batch_size = 1000
#     page = 0
#     print("Starting the movie update process...")
    
#     while True:
#         print(f"Processing batch {page} of movies...")
#         movies = get_movies_from_neo4j(batch_size=batch_size, page=page)
        
#         if not movies:
#             print("No more movies to process. Exiting...")
#             break  # Terminer quand il n'y a plus de films à traiter

#         for movie_record in movies:
#             movie_id = movie_record["movieId"]
            
#             # Récupérer les détails du film depuis l'API
#             movie_data = get_movie_details_from_movielens_api(movie_id)
#             if movie_data:
#                 # Mettre à jour le film dans Neo4j avec les champs spécifiques
#                 create_or_update_movie_in_neo4j(movie_data)
#                 # Créer les relations pour les acteurs et réalisateurs
#                 create_actors_and_directors_in_neo4j(movie_id, movie_data["actors"], movie_data["directors"])
#             else:
#                 print(f"Details not found or failed for movie ID {movie_id}")
        
#         page += 1
#         print(f"Batch {page} processed.")

# # Exécuter la mise à jour des films
# print("Starting movie update process...")
# update_movies_with_api_details()

# # Fermer la connexion Neo4j
# driver.close()
# print("Neo4j connection closed.")


Initializing Neo4j connection...
Neo4j connection initialized successfully.
Starting movie update process...
Starting the movie update process...
Processing batch 0 of movies...
Fetching movies from Neo4j, page 0 with batch size 1000...
Fetched 1000 movies from page 0.


KeyboardInterrupt: 

In [15]:
import requests
import csv

# Fonction pour récupérer tous les films depuis l'API MovieLens via l'exploration
def get_all_movies_from_movielens():
    auth_url = "https://movielens.org/api/sessions"
    auth_data = {"userName": "atf202", "password": "Bouboule12345"}
    
    session = requests.Session()
    response = session.post(auth_url, json=auth_data)

    if response.status_code == 200:
        print("Authenticated successfully with MovieLens API.")
        current_page = 1
        all_movies = []

        while True:
            movie_url = f"https://movielens.org/api/movies/explore?page={current_page}"
            movie_response = session.get(movie_url)

            if movie_response.status_code == 200:
                response_json = movie_response.json()

                # Vérification si la réponse contient une erreur
                if response_json.get("status") == "error" and "MLERR0" in response_json.get("message", ""):
                    print("MovieLens application error MLERR0 encountered. Stopping the process.")
                    break

                data = response_json.get("data", {}).get("searchResults", [])
                if not data:
                    print("No more movies to fetch. Process completed.")
                    break  # Fin des résultats

                all_movies.extend(data)
                current_page += 1
                print(f"Fetched page {current_page} with {len(data)} movies.")
            else:
                print(f"Failed to fetch movie details at page {current_page}. Status code: {movie_response.status_code}")
                break

        return all_movies
    else:
        print(f"Failed to authenticate with MovieLens API. Status code: {response.status_code}")
        return []

# Fonction pour écrire les détails des films dans un fichier CSV
def write_movies_to_csv(movies):
    print("Writing movies to CSV...")
    with open('movies2.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # En-têtes du fichier CSV
        writer.writerow(["movieId", "title", "genres", "imdbId", "tmdbId", "plotSummary", "releaseDate"])

        for movie in movies:
            movie_data = movie["movie"]
            writer.writerow([
                movie_data["movieId"],
                movie_data["title"],
                "|".join(movie_data["genres"]),
                movie_data["imdbMovieId"],
                movie_data["tmdbMovieId"],
                movie_data["plotSummary"],
                movie_data["releaseDate"]
            ])
    print("Movies written to CSV successfully.")

# Fonction pour écrire les images dans un fichier CSV
def write_images_to_csv(movies):
    print("Writing images to CSV...")
    with open('images.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # En-têtes du fichier CSV pour les images
        writer.writerow(["movieId", "posterPath", "backdropPath"])

        for movie in movies:
            movie_data = movie["movie"]
            poster_path = f"https://image.tmdb.org/t/p/original{movie_data['posterPath']}" if movie_data['posterPath'] else ""
            for backdrop in movie_data.get('backdropPaths', []):
                writer.writerow([
                    movie_data["movieId"],
                    poster_path,
                    f"https://image.tmdb.org/t/p/original{backdrop}"
                ])
    print("Images written to CSV successfully.")

# Fonction pour écrire les trailers dans un fichier CSV
def write_trailers_to_csv(movies):
    print("Writing trailers to CSV...")
    with open('trailers.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # En-têtes du fichier CSV pour les trailers
        writer.writerow(["movieId", "trailerId"])

        for movie in movies:
            movie_data = movie["movie"]
            for trailer in movie_data.get("youtubeTrailerIds", []):
                writer.writerow([movie_data["movieId"], trailer])
    print("Trailers written to CSV successfully.")

# Fonction pour écrire les acteurs dans un fichier CSV
def write_actors_to_csv(movies):
    print("Writing actors to CSV...")
    with open('actors.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # En-têtes du fichier CSV pour les acteurs
        writer.writerow(["movieId", "actor"])

        for movie in movies:
            movie_data = movie["movie"]
            for actor in movie_data.get("actors", []):
                writer.writerow([movie_data["movieId"], actor])
    print("Actors written to CSV successfully.")

# Fonction pour écrire les réalisateurs dans un fichier CSV
def write_directors_to_csv(movies):
    print("Writing directors to CSV...")
    with open('directors.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # En-têtes du fichier CSV pour les réalisateurs
        writer.writerow(["movieId", "director"])

        for movie in movies:
            movie_data = movie["movie"]
            for director in movie_data.get("directors", []):
                writer.writerow([movie_data["movieId"], director])
    print("Directors written to CSV successfully.")

# Fonction principale pour générer les fichiers CSV
def generate_csvs():
    print("Fetching all movies from MovieLens API...")
    all_movies = get_all_movies_from_movielens()
    if all_movies:
        write_movies_to_csv(all_movies)       # Pour les films
        write_images_to_csv(all_movies)       # Pour les images
        write_trailers_to_csv(all_movies)     # Pour les trailers
        write_actors_to_csv(all_movies)       # Pour les acteurs
        write_directors_to_csv(all_movies)    # Pour les réalisateurs

# Exécuter la génération des CSVs
print("Starting process to generate CSVs...")
generate_csvs()
print("CSV generation completed.")


Starting process to generate CSVs...
Fetching all movies from MovieLens API...
Authenticated successfully with MovieLens API.
Fetched page 2 with 24 movies.
Fetched page 3 with 24 movies.
Fetched page 4 with 24 movies.
Fetched page 5 with 24 movies.
Fetched page 6 with 24 movies.
Fetched page 7 with 24 movies.
Fetched page 8 with 24 movies.
Fetched page 9 with 24 movies.
Fetched page 10 with 24 movies.
Fetched page 11 with 24 movies.
Fetched page 12 with 24 movies.
Fetched page 13 with 24 movies.
Fetched page 14 with 24 movies.
Fetched page 15 with 24 movies.
Fetched page 16 with 24 movies.
Fetched page 17 with 24 movies.
Fetched page 18 with 24 movies.
Fetched page 19 with 24 movies.
Fetched page 20 with 24 movies.
Fetched page 21 with 24 movies.
Fetched page 22 with 24 movies.
Fetched page 23 with 24 movies.
Fetched page 24 with 24 movies.
Fetched page 25 with 24 movies.
Fetched page 26 with 24 movies.
Fetched page 27 with 24 movies.
Fetched page 28 with 24 movies.
Fetched page 29 wi

In [ ]:
from neo4j import GraphDatabase

# Connexion à Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"

print("Initializing Neo4j connection...")
driver = GraphDatabase.driver(uri, auth=(username, password))
print("Neo4j connection initialized successfully.")

# Fonction pour charger les films dans Neo4j
def load_movies():
    print("Loading movies into Neo4j...")
    query = """
    LOAD CSV WITH HEADERS FROM 'file:///movies2.csv' AS row
    MERGE (m:Movie {movieId: toInteger(row.movieId)})
    SET m.title = row.title,
        m.genres = row.genres,
        m.imdbId = row.imdbId,
        m.tmdbId = row.tmdbId,
        m.plotSummary = row.plotSummary,
        m.releaseDate = row.releaseDate
    """
    with driver.session() as session:
        session.run(query)
    print("Movies loaded into Neo4j.")

# Fonction pour charger les images dans Neo4j
def load_images():
    print("Loading images into Neo4j...")
    query = """
    LOAD CSV WITH HEADERS FROM 'file:///images.csv' AS row
    MERGE (m:Movie {movieId: toInteger(row.movieId)})
    MERGE (i:Image {posterPath: row.posterPath, backdropPath: row.backdropPath})
    MERGE (m)-[:HAS_IMAGE]->(i)
    """
    with driver.session() as session:
        session.run(query)
    print("Images loaded into Neo4j.")

# Fonction pour charger les trailers dans Neo4j
def load_trailers():
    print("Loading trailers into Neo4j...")
    query = """
    LOAD CSV WITH HEADERS FROM 'file:///trailers.csv' AS row
    MERGE (m:Movie {movieId: toInteger(row.movieId)})
    MERGE (t:Trailer {trailerId: row.trailerId})
    MERGE (m)-[:HAS_TRAILER]->(t)
    """
    with driver.session() as session:
        session.run(query)
    print("Trailers loaded into Neo4j.")

# Fonction pour charger les acteurs dans Neo4j
def load_actors():
    print("Loading actors into Neo4j...")
    query = """
    LOAD CSV WITH HEADERS FROM 'file:///actors.csv' AS row
    MERGE (a:Actor {name: row.actor})
    MERGE (m:Movie {movieId: toInteger(row.movieId)})
    MERGE (a)-[:ACTED_IN]->(m)
    """
    with driver.session() as session:
        session.run(query)
    print("Actors loaded into Neo4j.")

# Fonction pour charger les réalisateurs dans Neo4j
def load_directors():
    print("Loading directors into Neo4j...")
    query = """
    LOAD CSV WITH HEADERS FROM 'file:///directors.csv' AS row
    MERGE (d:Director {name: row.director})
    MERGE (m:Movie {movieId: toInteger(row.movieId)})
    MERGE (d)-[:DIRECTED]->(m)
    """
    with driver.session() as session:
        session.run(query)
    print("Directors loaded into Neo4j.")

# Fonction principale pour charger toutes les données
def load_data_into_neo4j():
    load_movies()       # Charger les films
    load_images()       # Charger les images
    load_trailers()     # Charger les trailers
    load_actors()       # Charger les acteurs
    load_directors()    # Charger les réalisateurs

# Exécution du chargement des données
print("Starting data load into Neo4j...")
load_data_into_neo4j()
print("Data loading completed.")

# Fermer la connexion Neo4j
driver.close()
print("Neo4j connection closed.")
